In [9]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import (GridSearchCV, StratifiedKFold,
                                     cross_validate, train_test_split)
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier

## Preprocessing  Pipeline
1. Lower case all column names
2. Columns to one hot encode:
    nominal_columns = ["highbp","highchol","cholcheck","smoker","stroke","heartdiseaseorattack","physactivity","fruits","veggies","hvyalcoholconsump","anyhealthcare","nodocbccost","diffwalk","sex"]
3. Columns to standardize:
    numerical_columns = ["bmi","age","income","menthlth","physhlth","education","genhlth"]
    



In [2]:
data = pd.read_csv("../data/raw/diabetes_binary_health_indicators_BRFSS2015.csv")
data.columns = data.columns.str.lower()

In [3]:
# Split data into train and test datasets. Cross validation will be performed on train dataset.
X = data.drop(columns=["diabetes_binary"])
y = data["diabetes_binary"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2025, shuffle=True, stratify=y)

In [4]:
# Building Preprocessing Pipeline Function
def create_pipeline(model: tuple[str, BaseEstimator])->Pipeline:
    nominal_columns = ["highbp","highchol","cholcheck","smoker","stroke","heartdiseaseorattack","physactivity","fruits","veggies","hvyalcoholconsump","anyhealthcare","nodocbccost","diffwalk","sex"]
    numerical_columns = ["bmi","age","income","menthlth","physhlth","education","genhlth"]

    column_trans = ColumnTransformer([
            ('numerical', StandardScaler(), numerical_columns),
            ('categorical', OneHotEncoder(drop='first', handle_unknown='ignore'), nominal_columns)
        ])

    pipe = Pipeline([("transformer", column_trans),model])
    return pipe

log_pipe = create_pipeline(('model', LogisticRegression(random_state=2024)))
rf_pipe = create_pipeline(('model', RandomForestClassifier(random_state=2024)))
svc_pipe = create_pipeline(('model', SVC(random_state=2024)))
knn = create_pipeline(('model', KNeighborsClassifier()))
gbc = create_pipeline(('model', GradientBoostingClassifier(random_state=2024)))
xgb = create_pipeline(('model', XGBClassifier(random_state=2024)))
xgb

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('numerical', StandardScaler(),
                                                  ['bmi', 'age', 'income',
                                                   'menthlth', 'physhlth',
                                                   'education', 'genhlth']),
                                                 ('categorical',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['highbp', 'highchol',
                                                   'cholcheck', 'smoker',
                                                   'stroke',
                                                   'heartdiseaseorattack',
                                                   'physactivity', 'fruits',
                                                   'veggies',
                                                   'hvyalcoholcon...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=2024, ...))])

In [5]:
# Split train dataset futher into train and validate dataset

X_train_dev, X_test_dev, y_train_validate, y_test_validate, = train_test_split(X_train, y_train, test_size=0.2, random_state=2025, shuffle=True, stratify=y_train)
print(X_train_dev.shape)

# Candidate Model Fit / Predict
rf_pipe.fit(X_train_dev, y_train_validate)
log_pipe.fit(X_train_dev, y_train_validate)
svc_pipe.fit(X_train_dev, y_train_validate)
knn.fit(X_train_dev, y_train_validate)
gbc.fit(X_train_dev, y_train_validate)
xgb.fit(X_train_dev, y_train_validate)

rf_pred = rf_pipe.predict(X_test_dev)
log_pred = log_pipe.predict(X_test_dev)
svc_pred = svc_pipe.predict(X_test_dev)
knn_pred = knn.predict(X_test_dev)
gbc_pred = gbc.predict(X_test_dev)
xgb_pred = xgb.predict(X_test_dev)

(182649, 21)


In [8]:
# Candidate Model Evaluation
def model_metrics(model, y_test, y_pred):

    # positive_label = '1'
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_area = roc_auc_score(np.array(y_test, dtype='float64'), np.array(y_pred, dtype='float64'))

    result_df = pd.DataFrame({'Model': [model], 'Accuracy': [accuracy], 'Precision': [precision], 'Recall': [recall], 'F1': [f1], 'ROC AUC': [roc_area]})
    return result_df

actual = y_test_validate.values

rf_metrics = model_metrics('Random Forest Classifier', actual, rf_pred)
log_metrics = model_metrics('Logistic Regression', actual, log_pred)
svc_metrics = model_metrics('Support Vector Classifier', actual, svc_pred)
knn_metrics = model_metrics('KNN Classifier', actual, knn_pred)
gbc_metrics = model_metrics('Gradient Boosting Classifier', actual, gbc_pred)
xgb_metrics = model_metrics('XGB Classifier', actual, xgb_pred)

results = pd.concat([rf_metrics, log_metrics, svc_metrics, knn_metrics, gbc_metrics, xgb_metrics]).reset_index(drop=True)
results

,Model,Accuracy,Precision,Recall,F1,ROC AUC
0,Random Forest Classifier,0.859405,0.487042,0.171330,0.253488,0.571060
1,Logistic Regression,0.863259,0.532240,0.153097,0.237793,0.565658
2,Support Vector Classifier,0.865208,0.642366,0.073405,0.131753,0.533394
3,KNN Classifier,0.847229,0.405770,0.207796,0.274844,0.579268
4,Gradient Boosting Classifier,0.866522,0.571890,0.166929,0.258426,0.573350
5,XGB Classifier,0.866018,0.561492,0.175102,0.266954,0.576483


In [13]:
# While accuarcy is relatively high at 84% minimum for all models, it is worth highlighting that there is significant class imbalance
# Create a dummy classifier that always predicts the most frequent class (all zeros in this case)
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train_dev, y_train_validate)
dummy_pred = dummy_clf.predict(X_test_dev)
dummy_metrics = model_metrics('Dummy Classifier (Most Frequent)', y_test_validate, dummy_pred)

# Add dummy classifier results to the overall results
results_with_dummy = pd.concat([dummy_metrics, results]).reset_index(drop=True).sort_values(by=["Accuracy","Precision"], ascending=False)
results_with_dummy

c:\Users\faisa\anaconda3\envs\mlserve\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Model,Accuracy,Precision,Recall,F1,ROC AUC
5,Gradient Boosting Classifier,0.866522,0.571890,0.166929,0.258426,0.573350
6,XGB Classifier,0.866018,0.561492,0.175102,0.266954,0.576483
3,Support Vector Classifier,0.865208,0.642366,0.073405,0.131753,0.533394
2,Logistic Regression,0.863259,0.532240,0.153097,0.237793,0.565658
0,Dummy Classifier (Most Frequent),0.860675,0.000000,0.000000,0.000000,0.500000
1,Random Forest Classifier,0.859405,0.487042,0.171330,0.253488,0.571060
4,KNN Classifier,0.847229,0.405770,0.207796,0.274844,0.579268


## Attempt to handle Label Data Imbalance